In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import lmfit


In [11]:
def exponential_decay(par, xdata):
    
    # Parse multiple input parameter
    # formats for intensity, rate    
    if hasattr(par,'valuesdict'):
        # lmfit parameter format
        var = par.valuesdict()
        inten = var['inten']
        rate = var['rate']
    elif hasattr(par,'keys'):
        # dict format
        inten = par['inten']
        rate = par['rate']
    else:
        # array/list/tuple format
        inten = par[0]
        rate = par[1]

    # Calculate the y-data from the parameters
    return inten * np.exp(-1*rate*xdata)

In [16]:
func = exponential_decay
groupcols = ['field']
xname = 'time'
yname = 'intensity'

method = 'leastsq'
sigma = 0.95
threads = 4

params = [{'name':'inten', 'value':np.asarray(data.groupby(groupcols)[yname].max()), 'vary':True},
          {'name':'rate', 'value':20.0, 'vary':True}]

In [17]:
data = pd.read_csv('GCN4_fourfield_R2.tsv', sep='\t')
data = data.loc[(data.resi==5)&(data.field<17.0)].reset_index(drop=True)

In [20]:
# pdlmfit __init__

input_data = data
func = func
groupcols = groupcols
params = params
xname = xname
yname = yname
yerr = None
method = method
sigma = sigma
threads = threads

index = ( data[groupcols + [xname]]
          .groupby(groupcols)
          .max()
          .index
         )
ngroups = index.shape[0]

In [22]:
params

[{'name': 'inten',
  'value': array([ 15647.389419,  68415.828125]),
  'vary': True},
 {'name': 'rate', 'value': 20.0, 'vary': True}]

In [45]:
params = [{'name':'inten', 'value':np.asarray(data.groupby(groupcols)[yname].max()), 'vary':True},
          {'name':'rate', 'value':20.0, 'vary':True}]

In [90]:
var_names = map(lambda x: x['name'], params)

# Expanded list of names and all properties to create a 
# multi-level column index
column_list = [(x['name'], y) 
               for x in params 
                   for y in x.keys()]

column_index = pd.MultiIndex.from_tuples(column_list, sortorder=None)

# Create a dataframe from the indexes and fill it
param_df = pd.DataFrame(index=index, columns=column_index)

for var in enumerate(param_df.columns.levels[0]):
    for key in param_df.loc[:,var[1]]:
        print (var[1], key, params[var[0]][key])
        param_df[(var[1],key)] = params[var[0]][key]

In [94]:
param_df.columns.levels[1]

Index([u'name', u'value', u'vary'], dtype='object')

In [98]:
param_df.loc[:,'inten']

,vary,name,value
field,,,
14.1,NaN,NaN,NaN
16.5,NaN,NaN,NaN


In [101]:
param_df = pd.DataFrame(index=index, columns=column_index)

for var in enumerate(param_df.columns.levels[0]):
    for key in param_df.loc[:,var[1]]:
        print (var[1], key, params[var[0]][key])
        param_df[(var[1],key)] = params[var[0]][key]

('inten', 'vary', True)
('inten', 'name', 'inten')
('inten', 'value', array([ 15647.389419,  68415.828125]))
('rate', 'vary', True)
('rate', 'name', 'rate')
('rate', 'value', 20.0)


In [102]:
param_df

inten                       rate            
       vary   name         value  vary  name value
field                                             
14.1   True  inten  15647.389419  True  rate  20.0
16.5   True  inten  68415.828125  True  rate  20.0

In [89]:
for col in column_index:
    print col

('inten', 'vary')
('inten', 'name')
('inten', 'value')
('rate', 'vary')
('rate', 'name')
('rate', 'value')


In [58]:
map(lambda x: x[0], column_list)

['inten', 'inten', 'inten', 'rate', 'rate', 'rate']

In [87]:
pd.concat([ pd.DataFrame(params[0]), pd.Series(params[1]).to_frame() ], axis=0)

,0,name,value,vary
0,NaN,inten,15647.389419,True
1,NaN,inten,68415.828125,True
name,rate,NaN,NaN,NaN
value,20,NaN,NaN,NaN
vary,True,NaN,NaN,NaN


In [85]:
param_df

inten                       rate            
       vary   name         value  vary  name value
field                                             
14.1   True  inten  15647.389419  True  rate  20.0
16.5   True  inten  68415.828125  True  rate  20.0

In [65]:
pd.DataFrame(params[1])

ValueError: If using all scalar values, you must pass an index

In [56]:
param_df

inten                       rate            
       vary   name         value  vary  name value
field                                             
14.1   True  inten  15647.389419  True  rate  20.0
16.5   True  inten  68415.828125  True  rate  20.0

MultiIndex(levels=[[u'inten', u'rate'], [u'name', u'value', u'vary']],
           labels=[[0, 0, 0, 1, 1, 1], [2, 0, 1, 2, 0, 1]])

In [47]:
param_df

inten                       rate            
       vary   name         value  vary  name value
field                                             
14.1   True  inten  15647.389419  True  rate  20.0
16.5   True  inten  68415.828125  True  rate  20.0

In [38]:
name_list

['inteninteninten', 'raterateraterate']

In [33]:
# pdlmfit fit
column_index.values

array([('inten', 'vary'), ('inten', 'name'), ('inten', 'value'),
       ('rate', 'a'), ('rate', 'vary'), ('rate', 'name'), ('rate', 'value')], dtype=object)

In [36]:
name_list

['inten', 'inten', 'inten', 'rate', 'rate', 'rate', 'rate']